In [ ]:
import pandas
import numpy
from pymongo import MongoClient
from nltk.tokenize import TweetTokenizer
from itertools import permutations
import time

In [ ]:
client = MongoClient("localhost", 27017)
job_table = client["job-search-database"]["jobs"]

In [ ]:
import re
link_pattern = re.compile('<a href[^<]+</a>')
noise_pattern = re.compile('^\W$')

In [ ]:
end_of_sentence = '<EOS>'

In [ ]:
def tokenize(job):
    job = job.lower()
    job = link_pattern.sub('', job)
    job = (
        job.replace('<p>', '')
        .replace('&#x27;', "'")
        .replace('&quot;', '"')
        .replace('|', '')
    )
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(job)
    tokens = [ t if t != '.' else end_of_sentence for t in tokens ] # For synthetic data shuffling
    tokens = [ t for t in tokens if should_keep_token(t) ]
    return tokens

In [ ]:
def should_keep_token(token):
    return (
        re.match(noise_pattern, token) == None and
        not token.startswith('@') and
        not token.startswith('#')
    )

In [ ]:
def process(data):
    data.drop(['_id', 'by', 'id', 'parent', 'date'], axis=1, inplace=True)
    data = data[data['text'].isnull() == False]
    data = data[data['preferred'].isnull() == False]
    data['preferred'] = data['preferred'].map(lambda x: 1 if x else 0)
    data['tokens'] = data['text'].map(tokenize)
    data = data[data.tokens != 'NC']
    data = data[data.tokens.apply(lambda x: len(x) > 20)]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    return data

In [ ]:
preferred = pandas.DataFrame(job_table.find({ '$and': [{ 'preferred': { '$exists': True } }, { 'text': { '$exists': True } }] }))

In [ ]:
processed = process(preferred)

In [ ]:
samples = processed[processed['preferred'] == 1]['tokens']

In [ ]:
samples.reset_index(drop=True, inplace=True)

In [ ]:
def find_split(i, available_splits, total_splits):
    return ((i + 1) * max((available_splits // total_splits), 1)) - 1

In [ ]:
def group_sentences(tokens, max_splits = 2):
    av_splits = []
    for i in range(len(tokens)):
        if tokens[i] == end_of_sentence and i != 0 and i != (len(tokens) - 1):
            av_splits.append(i)

    collection = []
    total_splits = min(max_splits, len(av_splits))
    calc_split = lambda x: find_split(x, len(av_splits), total_splits)

    if total_splits == 0:
        collection.append(tokens)
    else:
        for i in range(total_splits + 1): # total groups is 1 more than total splits
            if i == 0:
                collection.append(tokens[:av_splits[calc_split(i)]])
            elif i == total_splits:
                collection.append(tokens[av_splits[calc_split(i - 1)]:])
            else:
                collection.append(tokens[av_splits[calc_split(i - 1)]: av_splits[calc_split(i)]])

    return collection
        

In [ ]:
def flatten(arr):
    return [ x for sub in arr for x in sub ]

In [ ]:
def time_perm(n):
    start = time.time()
    perms = list(permutations(grouped[n]))
    print(f"{n} took {time.time() - start} sec")
    return perms

In [ ]:
group_sentences(samples[0])

In [ ]:
grouped = [ group_sentences(tokens) for tokens in samples ]

In [ ]:
p = list(permutations(grouped[0]))
len(p)

In [ ]:
expanded = []
for group in grouped:
    for x in permutations(group):
        my_groups.append(flatten(x))

In [ ]:
len(my_groups)

In [ ]:
o = flatten(p)

In [ ]:
len(o)

In [ ]:
expanded = []
for i in range(len(grouped)):
    expanded.append()

In [ ]:
max_len = 0
j = -1
for i in range(34):
    if len(grouped[i]) > max:
        max_len = len(grouped[i])
        j = i
max, j

In [ ]:
time_perm(0)

In [ ]:
synthesized = flatten([ list(permutations(group)) for group in grouped ])

In [ ]:
l = [ list(permutations(g)) for g in grouped ]

In [ ]:
flatten

In [ ]:
samples

In [ ]:
len([ x for x in samples if sentence_key in x ])